In [0]:
# mount the drive to access to data files
from google.colab import drive
drive.mount('drive')

In [0]:
import keras

from keras import backend as K

from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image

from keras.models import Sequential
from keras.models import Model

from keras.layers import Dense, Dropout, Activation, Flatten, ActivityRegularization, Lambda
from keras.layers import Convolution2D, MaxPooling2D, Conv1D, MaxPooling1D, TimeDistributed
from keras.layers import AveragePooling2D, Input

from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input

from keras.utils import np_utils, normalize
from keras.engine import InputLayer

################################################################################
import sklearn

from sklearn.preprocessing import normalize
from sklearn.linear_model import SGDClassifier

################################################################################
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

import os
import h5py
import glob
import time

################################################################################

def read_dir(dir_path) :
  """return the differents categories, the images in these categories, and the number of images per category"""
  listDir = sorted(os.listdir(dir_path))
  sizes = []
  listFiles = []
  for d in listDir :
    files = sorted(os.listdir(dir_path+'/'+d))
    sizes += [len(files)]
    listFiles += [files]
  return listDir, listFiles, sizes


def create_zero_array(layer, n, nb_images):
  """return a numpy array with zeros and with the needed scales"""
  if layer == 'block5_pool' :
    X = np.zeros((nb_images, 7 + 10*n, 7 + 10*n, 512))
  else :
    if n == 0 :
      X = np.zeros((nb_images, 4096))
    else :
      X = np.zeros((nb_images, -3 + 5*n, -3 + 5*n, 4096))
  return X
  

def complete_array(dir_path, dir_save, listDir, listFiles, X, n) :
  """for all the images in listFiles, extract the features thanks to the network"""
  """return a 4-dimensional array with a 3-dimensional array per image"""
  X_new = X
  nb = 0
  cpt = 0
  t0 = time.time()
  for i in range (len(listDir)) :
    d = listDir[i]
    files = listFiles[i]
    print(d, len(files))
    nb += len(files)
    for f in files :
      t1 = time.time()
      img = image.load_img(dir_path+'/'+d+'/'+f, target_size=(224 + 320*n, 224 + 320*n))
      x = image.img_to_array(img)
      x = np.expand_dims(x, axis=0)
      x = preprocess_input(x)
      features = network2.predict(x)
      X_new[cpt,:] = features[0,:]
      cpt += 1
      t2 = time.time()
      print(str(cpt) + '/' + str(nb), t2 - t1)
    np.save(dir_save, X_new)
  print('total', t2 - t0)
  return X_new

def pool(X, p):
  """return the numpy array X on which the pooling p was performed"""
  if p == 'max' :  
    X_pool = np.max(X, axis = (1, 2)) 
  elif p == 'mean' :
    X_pool = np.mean(X, axis = (1, 2)) 
  else :
    X_pool = X
  return X_pool


def to_fully_conv(model):
  """transforms the Convolutional Neural Network model into a Fully Convolutional Network"""

    new_model = Sequential()

    input_layer = InputLayer(input_shape=(None, None, 3), name="input_new")

    new_model.add(input_layer)

    for layer in model.layers:

        if "Flatten" in str(layer):
            flattened_ipt = True
            f_dim = layer.input_shape

        elif "Dense" in str(layer):

            input_shape = layer.input_shape
            output_dim =  layer.get_weights()[1].shape[0]
            W,b = layer.get_weights()

            if flattened_ipt:
                shape = (f_dim[1],f_dim[2],f_dim[3],output_dim)
                new_W = W.reshape(shape)
                new_layer = Convolution2D(output_dim,
                                          (f_dim[1],f_dim[2]),
                                          strides=(1,1),
                                          activation=layer.activation,
                                          padding='valid',
                                          weights=[new_W,b])
                flattened_ipt = False

            else:
                shape = (1,1,input_shape[1],output_dim)
                new_W = W.reshape(shape)
                new_layer = Convolution2D(output_dim,
                                          (1,1),
                                          strides=(1,1),
                                          activation=layer.activation,
                                          padding='valid',
                                          weights=[new_W,b])


        else:
            new_layer = layer

        new_model.add(new_layer)

    return new_model
  


      
print('Done')

In [0]:


#layer from which we take the output : block5_pool, fc1 or fc2
layer = 'block5_pool'

#name of the document where the train and test set are stocked
doc = ''

#scale of images = (224 + 320n)*(224+320n)
n = 0

#pooling performed on the output : max, mean, or not
pooling = 'mean'


#definition of the network used (VGG19)
network = VGG19(weights='imagenet')#, input_shape=(fscale, fscale, 3))
network2 = Model(inputs=network.input, outputs=network.get_layer(layer).output)
if n >= 1 : 
  network2 = to_fully_conv(network2)

################################################################################

"""ARTIFICIAL NEURAL NETWORK"""

#path where train images and test images are stocked
dir_path_train = 'drive/My Drive/Datasets/' + doc + '/train'
dir_path_test = 'drive/My Drive/Datasets/'+doc+'/test'

#path where the output array will be saved
save_train = 'drive/My Drive/Datasets/' + doc + '/X_train_' + layer + '_' + pooling + '(' + str(n) + ').npy'
save_test = 'drive/My Drive/Datasets/' + doc + '/X_test_' + layer + '_' + pooling + '(' + str(n) + ').npy'

#creating numpy array from images
listDir_train, listFiles_train, sizes_train = read_dir(dir_path_train)
listDir_test, listFiles_test, sizes_test = read_dir(dir_path_test)

X_train = create_zero_array(layer, n, sum(sizes_train))
X_test = create_zero_array(layer, n, sum(sizes_test))

X_train = complete_array(dir_path_train, save_train, listDir_train, listFiles_train, X_train, n)
X_test = complete_array(dir_path_test, save_test, listDir_test, listFiles_test, X_test, n)

#pooling
X_train = pool(X_train, pooling)
X_test = pool(X_test, pooling)

#normalization
X_train = sklearn.preprocessing.normalize(X_train, norm = 'l2', axis = 1)
X_test = sklearn.preprocessing.normalize(X_test, norm = 'l2', axis = 1)


"""SUPPORT VECTOR MACHINE"""

print(X_train.shape, X_test.shape)

parameters = {'C':[0.0001,0.001,0.01, 0.1, 1, 10,100]}
svr = svm.SVC(kernel='linear')
clf = GridSearchCV(svr, parameters, cv=3)
clf.fit(X_train, Y_train)

print('Accuracy: %.2f' % clf.score(X_test, Y_test))

print('Done')

